In [ ]:
# This notebook shows how to make use of the goodreads co-occurence rating matrix
# Please, download the files 'user_book_matrix.npy' and 'user_information.p' from the drive 
# and place them in a folder named "goodreads".

# Disclaimer: goodreads books were matched with dmc books via fuzzy string match of title and author, 
# some ill-matched cases are possible

In [1]:
import pickle
import sys
import numpy as np
from scipy.sparse import csr_matrix, load_npz
import csv
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from evaluation_workflow import evaluate_recommender

#### Load DMC Data

In [ ]:
dmc_task_books = dict()
with open("DMC-2021-Task/items.csv") as i:
    csvreader = csv.reader(i,delimiter="|")
    next(csvreader) # header
    for line in csvreader:
        itemID, title = line[:2]
        dmc_task_books[int(itemID)] = title

print(f"example: {list(dmc_task_books.items())[0]}")
print(f"all task books: {len(dmc_task_books)}")

#### Load User Information
-> Dictionary of userID and value-list with "avg rating" and "no. of rated book" per user

In [ ]:
with open("goodreads/user_information.p", "rb") as u:
    user_information = pickle.load(u)

user_information[0]

#### Load DMC Books that are on Goodreads
and exclude those, that were clearly matched incorrectly with a one-word-title from goodreads

In [ ]:
from book_classes import DMCBook, GoodreadsBook, GoodreadsAuthor
#from book_classes import DMCGenre

dmc_to_gr_string = pickle.load(open("goodreads/dmc_to_goodreads.p", "rb"))
dmc_to_gr = {int(k): int(v) for k,v in dmc_to_gr_string.items()}
one_word_titles = {}
genres = ["children_with_matches.p",
          "comics_graphic_with_matches.p",
          "dmc_remaining_with_matches.p",
          "fantasy_paranormal_with_matches.p",
          "history_biography_with_matches.p",
          "mystery_thriller_crime_with_matches.p",
          "romance_with_matches.p",
          "young_adult_with_matches.p"]
for gf in genres:
    with open(f"goodreads/matched/{gf}", "rb") as g:
        try:
            dmc_books = pickle.load(g)
            for dmcb in dmc_books:
                if dmcb.goodreads_match != None:
                    gr_title = dmcb.goodreads_match.title
                    if len(gr_title) == 1 and len(dmcb.title) != 1:
                        one_word_titles[dmcb.itemID] = (dmcb.title, gr_title)
        except EOFError:
            #print(f"empty file: {gf}\n")
            pass

print(f"one word title matches that get excluded: {len(one_word_titles)}\n")

for item in one_word_titles.keys():
        del dmc_to_gr[item]

print(f"all task books on goodreads: {len(dmc_to_gr)}")

#### Load Book-User-Matrix

In [ ]:
user_book_matrix = np.load("goodreads/user_book_matrix.npy", allow_pickle=True)
#user_book_matrix = load_npz("goodreads/user_book_matrix.npy")

user_book_matrix = user_book_matrix.item()
print(user_book_matrix.shape)
print(type(user_book_matrix[0,:]))
print(user_book_matrix.count_nonzero())

In [ ]:
# retrieve relevant book columns
book_columns = dict()
coo_matrix = user_book_matrix.tocoo()
ratings_non_zero = set(zip(coo_matrix.row, coo_matrix.col))
for book,user in ratings_non_zero:
    if int(book) in dmc_to_gr.keys():
        book_columns[int(book)] = user_book_matrix[book,:]

In [ ]:
# DMC books with user ratings
len(book_columns)

In [ ]:
def compute_similarity(book_a, book_b):
    return 1 - distance.cosine(book_a, book_b)

def compute_top10_recs(itemID, book_columns):
    similarity = {}
    book_vector = book_columns[itemID].toarray()[0]
    for book_id, ratings in book_columns.items():
        sim = compute_similarity(book_vector, ratings.toarray()[0].T)
        similarity[book_id] = sim
    top10 = list(dict(sorted(similarity.items(), key=lambda item: item[1], reverse=True)).items())[1:11]
    top10_books = [(b[0], dmc_task_books[b[0]],b[1]) for b in top10]
    return top10_books
    
# todo: add normalization (subtract average user rating for each user in two vectors of interest)

In [ ]:
last_dragon = 13834
fire_and_ice = 54197
orange_girl = 39249
more_happy_than_not = 8791
wizard_oz = 40426

print(compute_top10_recs(last_dragon, book_columns))
print("\n")
"""print(compute_top10_recs(fire_and_ice, book_columns))
print("\n")
print(compute_top10_recs(orange_girl, book_columns))
print("\n")
print(compute_top10_recs(more_happy_than_not, book_columns))
print("\n")
print(compute_top10_recs(wizard_oz, book_columns))"""

### Load amazon validation and test set and check how much books are covered on goodreads

In [2]:
df_val = pd.read_csv('dmc21_amazon_validation.csv')
validation_books = list(df_val["itemID"])
df_test = pd.read_csv('dmc21_amazon_test.csv')
test_books = list(df_test["itemID"])
if df_test["rec1_ID"].isnull().values.any():
    print(df_test["rec1_ID"].isna().sum())

71


In [ ]:
# compute predictions for covered validation books
df_val_covered = df_val[df_val['itemID'].isin(book_columns.keys())]

books_val_with_preds = dict()
c = 0
for bookID in df_val_covered["itemID"]:
    books_val_with_preds[bookID] = compute_top10_recs(bookID, book_columns)
    c += 1
    print(f"done with {c}", end="\r")

with open("amazon_validation_goodreads_predictions", "wb") as f:
    pickle.dump(books_val_with_preds,f)

In [3]:
# compute metric for all/covered validation books

with open("amazon_validation_goodreads_predictions", "rb") as f:
    books_val_with_preds = pickle.load(f)

preds_dict_goodreads = dict()
preds_dict_all = {k: [] for k in df_val["itemID"].values}
for b, recs in books_val_with_preds.items():
    recs = [int(r[0]) for r in recs]
    preds_dict_goodreads[b] = recs
    if b in preds_dict_all.keys():
        preds_dict_all[b] = recs

In [7]:
from statistics import mean

def precision_at_k(y_true, y_pred, k):
    """
    Relevancy of items in top-k predicted recommendations.
    For cases with no predicted recommendations, precision is automatically 1.
    ! Order un-aware metric.
    """
    if len(y_pred) == 0:
        return 1
    else:
        y_pred_at_k = y_pred[:k]
        tp = 0
        fp = 0
        for pred in y_pred_at_k:
            if pred in y_true:
                tp += 1
            else:
                fp += 1
        precision_at_k = tp / (tp + fp)
        return precision_at_k


def recall_at_k(y_true, y_pred, k):
    """
    Coverage of relevant items in top-k predicted recommendations.
    For cases with no predicted recommendations, recall is automatically 0.
    ! Order un-aware metric.
    """
    if len(y_pred) == 0:
        return 0
    else:
        y_pred_at_k = y_pred[:k]
        tp = 0
        fn = 0
        for true in y_true:
            if true in y_pred_at_k:
                tp += 1
            else:
                fn += 1
        recall_at_k = tp / (tp + fn)
        return recall_at_k


def f1_score_at_k(precision_at_k, recall_at_k):
    """
    F1 score for k predictions.
    ! Order un-aware metric.
    """
    if precision_at_k == 0 and recall_at_k == 0:
        return 0
    else:
        return (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)


def avg_precision(y_true, y_preds):
    correct_preds = 0
    running_sum = 0
    for k in range(len(y_preds)):
        if y_preds[k] in y_true:
            correct_preds += 1
            running_sum += correct_preds / (k + 1)
    avg_precision = running_sum / len(y_true)
    return avg_precision

class ValidationBook:
    def __init__(self, b_id, recs):
        self.b_id = b_id
        self.recs = recs


def parse_eval_set(filename):
    df_val = pd.read_csv(filename)
    val_books = []
    for index, row in df_val.iterrows():
        b_id = row[0]
        recs = []
        for rec in row[1:]:
            if isinstance(rec, str):
                rec_splitted = rec.split()
                if len(rec_splitted) > 1:
                    recs.extend([int(r) for r in rec_splitted])
                else:
                    recs.append(int(rec))
        if len(recs) > 0:
            val_books.append(ValidationBook(b_id, recs))

    print(
        f"Number of books with recommendations in validation set: {len(val_books)} of {len(df_val)}"
    )
    print(
        f"Average number of recommendations per book in validation set: {round(mean([len(v.recs) for v in val_books]),1)}"
    )

    return val_books

def compute_metrics(val_books, predictions_dict):
    for k in [1, 2, 3, 5, 6, 7]:
        precision_at_k_values = []
        recall_at_k_values = []
        f1_at_k_values = []

        if k == 1:
            avg_precision_values = []

        for vb in val_books:
            p = precision_at_k(vb.recs, predictions_dict[vb.b_id], k)
            r = recall_at_k(vb.recs, predictions_dict[vb.b_id], k)
            f1 = f1_score_at_k(p, r)
            precision_at_k_values.append(p)
            recall_at_k_values.append(r)
            f1_at_k_values.append(f1)

            if k == 1:
                ap = avg_precision(vb.recs, predictions_dict[vb.b_id])
                avg_precision_values.append(ap)

        # compute average across validation set
        p_at_k = round(mean(precision_at_k_values) * 100, 2)
        r_at_k = round(mean(recall_at_k_values) * 100, 2)
        f1_at_k = round(mean(f1_at_k_values) * 100, 2)
        mean_avg_precision = round(mean(avg_precision_values) * 100, 2)

        print(f"Precision@{k}: {p_at_k}%")
        print(f"Recall@{k}: {r_at_k}%")
        print(f"F1-Measure@{k}: {f1_at_k}%")
        print("-------")

    print(f"Mean Average Precision: {mean_avg_precision}%")

print("For validation books that are on goodreads:\n")
val_books = parse_eval_set('dmc21_amazon_validation.csv')
val_books_goodreads = [vb for vb in val_books if vb.b_id in preds_dict_goodreads.keys()]
compute_metrics(val_books_goodreads,preds_dict_goodreads)

For validation books that are on goodreads:

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 4.0%
Recall@1: 0.62%
F1-Measure@1: 1.07%
-------
Precision@2: 6.0%
Recall@2: 2.9%
F1-Measure@2: 3.69%
-------
Precision@3: 4.67%
Recall@3: 3.4%
F1-Measure@3: 3.73%
-------
Precision@5: 3.6%
Recall@5: 4.69%
F1-Measure@5: 3.81%
-------
Precision@6: 3.67%
Recall@6: 5.27%
F1-Measure@6: 4.05%
-------
Precision@7: 3.14%
Recall@7: 5.27%
F1-Measure@7: 3.7%
-------
Mean Average Precision: 2.97%


In [4]:
evaluate_recommender(preds_dict_all,val_set="dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 79.31%
Recall@1: 0.13%
F1-Measure@1: 0.23%
-------
Precision@2: 79.74%
Recall@2: 0.63%
F1-Measure@2: 0.8%
-------
Precision@3: 79.45%
Recall@3: 0.73%
F1-Measure@3: 0.8%
-------
Precision@5: 79.22%
Recall@5: 1.01%
F1-Measure@5: 0.82%
-------
Precision@6: 79.24%
Recall@6: 1.14%
F1-Measure@6: 0.87%
-------
Precision@7: 79.13%
Recall@7: 1.14%
F1-Measure@7: 0.8%
-------
Mean Average Precision: 0.64%
